<a href="https://colab.research.google.com/github/SHKajal/RPi/blob/main/CNN_2_Binary_col_586.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/CCIC-CIC-AndMal-2020/all_cat_586_col_Label.csv')

In [ ]:
df

In [ ]:
y = df.Category

In [ ]:
X = df.drop('Category', axis=1,inplace=False)

In [ ]:
del df

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler().fit(X)
X = scaler.transform(X)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=.50)

In [ ]:
del X
del y

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Convolution1D, MaxPooling1D

In [ ]:
model = Sequential()

model.add(Convolution1D(64, kernel_size=12, padding="same",activation="relu",input_shape=(X_train.shape[1], 1))) # df.shape = (105692, 9504)
model.add(MaxPooling1D(pool_size=(5)))
model.add(BatchNormalization())

model.add(Convolution1D(128, kernel_size=6, padding="same",activation="relu"))
model.add(MaxPooling1D(pool_size=(3)))
model.add(BatchNormalization())

model.add(Convolution1D(128, kernel_size=3, padding="same",activation="relu"))
model.add(MaxPooling1D(pool_size=(2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(10, activation='relu'))
# model.add(Dense(6, activation='relu'))
# model.add(Dense(2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.load_weights("/content/drive/MyDrive/Models/CNN_2_Binary_col_587/weights/50.h5")

In [ ]:
import tensorflow as tf
model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Nadam(0.0001),
    metrics=['binary_accuracy']
)

In [ ]:
saving_path = '/content/drive/MyDrive/Models/CNN_2_Binary_col_587'

In [ ]:
checkpoint_path = saving_path + "/weights/{epoch:02d}.h5"
from keras.callbacks import ModelCheckpoint
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=False,
    save_best_only=False,
    monitor='val_accuracy',
    mode='max',
    save_freq='epoch',
    verbose=0)

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[checkpoint_callback])

In [ ]:
from sklearn.metrics import classification_report
y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred.round(), digits=4)
print(report)

In [ ]:
with open(saving_path + "/history.txt", 'w') as file:
    file.write(str(history.history))

with open(saving_path + "/report.txt", 'w') as file:
    file.write(str(report))


In [ ]:
import matplotlib.pyplot as plt

# Extract the training and validation metrics from the history object
train_acc = history.history['binary_accuracy']
val_acc = history.history['val_binary_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Create a figure with subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 10))

# Plot training and validation accuracy
ax1.plot(train_acc, label='Train Accuracy')
ax1.plot(val_acc, label='Validation Accuracy')
ax1.set_title('Accuracy')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()

# Plot training and validation loss
ax2.plot(train_loss, label='Train Loss')
ax2.plot(val_loss, label='Validation Loss')
ax2.set_title('Loss')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()

# Adjust spacing between subplots
fig.tight_layout()

# Display the figure
plt.show()
